In [2]:
# !git clone https://github.com/NGUYENKHANH-UT/Vietnamese-Scenetext-Recognition.git



%cd Vietnamese-Scenetext-Recognition

/home/ttd/hcmai/aic2024/multimedia-retrieval/inference/Vietnamese-Scenetext-Recognition


In [3]:
!pip install -r requirements.txt

In [4]:
!pip install kaggle

In [5]:

# !mkdir Keyframes
# %cd Keyframes
# !kaggle datasets download -d nguyenthanhkhanh/one-keyframes
# !unzip -q one-keyframes.zip
# !rm -rf one-keyframes.zip

In [6]:

%cd src/craft
!mkdir weights
%cd weights
!gdown --id 1Jk4eGD7crsqCCg9C9VjCLkMN3ze8kutZ
!gdown --id 1XSaFwBkOaFOdtk4Ane3DFyJGPRw6v5bO
%cd ../../..

/home/ttd/hcmai/aic2024/multimedia-retrieval/inference/Vietnamese-Scenetext-Recognition/src/craft
mkdir: cannot create directory ‘weights’: File exists
/home/ttd/hcmai/aic2024/multimedia-retrieval/inference/Vietnamese-Scenetext-Recognition/src/craft/weights
/home/ttd/anaconda3/envs/test_ocr/lib/python3.12/site-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Jk4eGD7crsqCCg9C9VjCLkMN3ze8kutZ
To: /home/ttd/hcmai/aic2024/multimedia-retrieval/inference/Vietnamese-Scenetext-Recognition/src/craft/weights/craft_mlt_25k.pth
100%|██████████████████████████████████████| 83.2M/83.2M [00:04<00:00, 17.3MB/s]
/home/ttd/anaconda3/envs/test_ocr/lib/python3.12/site-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to u

In [7]:
import numpy as np
from PIL import Image
import cv2
import os
import json
import glob
from tqdm import tqdm
from types import SimpleNamespace

# Import custom modules
from src.utils.four_points_transform import four_points_transform
from src.utils.encode_base64 import encode_base64

from src.craft.craft_predict import predict_craft, str2bool
from src.vietocr.vietocr_predict import predict_vietocr

from src.craft.load_model import load_model_craft
from src.vietocr.load_model import load_model_vietocr

from src.utils.group_text_box import group_text_box
from src.craft.craft import CRAFT


In [8]:
args = SimpleNamespace(
    trained_model='src/craft/weights/craft_mlt_25k.pth',
    text_threshold=0.7,
    low_text=0.4,
    link_threshold=0.4,
    cuda=True,  # Nếu bạn có GPU và muốn sử dụng, hãy thay đổi thành True
    canvas_size=1280,
    mag_ratio=1.5,
    poly=False,
    show_time=False,
    refine=False,
    refiner_model='src/craft/weights/craft_refiner_CTW1500.pth'
)


In [9]:
keyframes_dir = './Keyframes'
all_keyframe_paths = dict()
for part in sorted(os.listdir(keyframes_dir)):
    #data_part = part.split('_')[-1] # L01, L02 for ex
    all_keyframe_paths[part] =  dict()

for data_part in sorted(all_keyframe_paths.keys()):
    data_part_path = f'{keyframes_dir}/{data_part}'
    video_dirs = sorted(os.listdir(data_part_path))
    video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
    for video_id, video_dir in zip(video_ids, video_dirs):
        keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
        all_keyframe_paths[data_part][video_id] = keyframe_paths

In [10]:
net, refine_net = load_model_craft(args=args)
detector = load_model_vietocr()

/home/ttd/anaconda3/envs/test_ocr/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ttd/anaconda3/envs/test_ocr/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loading weights from checkpoint (src/craft/weights/craft_mlt_25k.pth)


/home/ttd/hcmai/aic2024/multimedia-retrieval/inference/Vietnamese-Scenetext-Recognition/src/craft/load_model.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_st

Model weight /tmp/vgg_transformer.pth exsits. Ignore download!


In [12]:
from timeit import default_timer as timer

save_dir = './ocr'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

keys = sorted(all_keyframe_paths.keys())
for key in tqdm(keys):
    video_keyframe_paths = all_keyframe_paths[key]
    video_ids = sorted(video_keyframe_paths.keys())

    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))

    video_ocr_results, image_ocr_result = [], []

    for video_id in tqdm(video_ids):
        video_keyframe_path = video_keyframe_paths[video_id]
        video_ocr_results = []

        for i in range(len(video_keyframe_path)):
            s = timer()
            
            image_path = video_keyframe_path[i]
            main_image = cv2.imread(image_path)
            boxes = predict_craft(net, refine_net, image_path=image_path, args=args)
            boxes = group_text_box(boxes)

            # rec phase
            for count, box in enumerate(boxes):
                image_ocr_result = []
                # cat anh ra va convert to PIL
                sub_img = four_points_transform(main_image, np.array(box, dtype='float32'))
                # write content
                box = [list(map(int, i)) for i in box]
                box = [list(map(lambda x: max(x, 0) ,i)) for i in box]


                # ap dung can bang sang
                # sub_img = histogram_equalzed_rgb(sub_img)
                sub_img = cv2.cvtColor(sub_img, cv2.COLOR_BGR2RGB)
                sub_img = Image.fromarray(sub_img)

                # predict with vietocr
                (pred, prob) = predict_vietocr(detector, sub_img)

                # write to txt
                if prob > 0.5:
                  image_ocr_result.append(pred)

            video_ocr_results.append(image_ocr_result)
            
            e = timer()
            print(f'Finished ocr frame in {e-s} seconds')
            
              
        with open(f'{save_dir}/{key}/{video_id}.json',"w", encoding='utf-8') as jsonfile:
            
            json.dump(video_ocr_results, jsonfile, ensure_ascii=False)

  0%|          | 0/1 [00:00<?, ?it/s]

Finished ocr frame in 0.4661090140000397 seconds
Finished ocr frame in 0.24265417800006617 seconds
Finished ocr frame in 0.12576654999998027 seconds
Finished ocr frame in 1.094475599999896 seconds
Finished ocr frame in 0.6559753480000836 seconds
Finished ocr frame in 1.258808596000108 seconds
Finished ocr frame in 1.0131494949998796 seconds
Finished ocr frame in 0.3985676700001477 seconds
Finished ocr frame in 0.38596577099997376 seconds
Finished ocr frame in 0.47991380000007666 seconds
Finished ocr frame in 0.47592541100016206 seconds
Finished ocr frame in 0.41623630000003686 seconds
Finished ocr frame in 0.5137243230001332 seconds
Finished ocr frame in 0.4647799639999448 seconds
Finished ocr frame in 0.4528819229999499 seconds
Finished ocr frame in 1.1302315129998988 seconds
Finished ocr frame in 0.2723766709998472 seconds
Finished ocr frame in 0.5851309160000255 seconds
Finished ocr frame in 1.1859638450000602 seconds
Finished ocr frame in 0.9343596090000119 seconds
Finished ocr fra

In [ ]:
# from timeit import default_timer as timer

# batch_size = 512  # You can set this to your desired batch size

# save_dir = './ocr'
# if not os.path.exists(save_dir):
#     os.mkdir(save_dir)

# keys = sorted(all_keyframe_paths.keys())
# for key in tqdm(keys):
#     video_keyframe_paths = all_keyframe_paths[key]
#     video_ids = sorted(video_keyframe_paths.keys())

#     if not os.path.exists(os.path.join(save_dir, key)):
#         os.mkdir(os.path.join(save_dir, key))

#     for video_id in tqdm(video_ids):
#         video_keyframe_path = video_keyframe_paths[video_id]

#         sub_img_list = []
#         boxes_len = []
        
#         craft_s = timer()
#         for i in range(len(video_keyframe_path)):
#             image_path = video_keyframe_path[i]
#             main_image = cv2.imread(image_path)

#             boxes = predict_craft(net, refine_net, image_path=image_path, args=args)
#             boxes = group_text_box(boxes)
            
#             boxes_len.append(len(boxes))

#             for count, box in enumerate(boxes):
#                 # Transform the image and convert to PIL
#                 sub_img = four_points_transform(main_image, np.array(box, dtype='float32'))
#                 sub_img = cv2.cvtColor(sub_img, cv2.COLOR_BGR2RGB)
#                 sub_img = Image.fromarray(sub_img)
#                 sub_img_list.append(sub_img)

#         craft_e = timer()
#         print(len(sub_img_list))
#         print(f'Craft time: {craft_e - craft_s}')
        
        
#         total_length = len(sub_img_list)
#         num_batches = (total_length + batch_size - 1) // batch_size
#         print('Num batches = ', num_batches)
#         ocr_results = []
#         probs = []
        
#         vietocr_s = timer()
#         for batch_idx in range(num_batches):
#             batch_start = batch_idx * batch_size
#             batch_end = min((batch_idx + 1) * batch_size, total_length)
#             batch_images = sub_img_list[batch_start:batch_end]
            
#             (pred, prob) = predict_vietocr(detector, batch_images, True)
            
#             # Truncate the results to the actual batch size
#             pred = pred[:batch_end - batch_start]
#             prob = prob[:batch_end - batch_start]
            
#             ocr_results.extend(pred)
#             probs.extend(prob)
        
#         vietocr_e = timer()
#         print(f'VietOCR time: {vietocr_e - vietocr_s}')
        
#         video_ocr = []
#         prob_results = []   
#         start = 0
#         for length in boxes_len:
#             end = start + length
#             video_ocr.append(ocr_results[start:end])
#             prob_results.append(probs[start:end])
#             start = end
        
#         video_ocr_results = []
#         for pred, prob in zip(video_ocr, prob_results):
#             res = [pred[i] for i in range(len(pred)) if prob[i] > 0.5]
#             video_ocr_results.append(res)
        
#         with open(f'{save_dir}/{key}/{video_id}.json',"w", encoding='utf-8') as jsonfile:
#             json.dump(video_ocr_results, jsonfile, ensure_ascii=False)

  0%|          | 0/1 [00:00<?, ?it/s]

2246
Craft time: 29.713171440991573
Num batches =  5


100%|██████████| 1/1 [03:10<00:00, 190.03s/it]

VietOCR time: 149.639736290992
